In [1]:
import matplotlib
matplotlib.use('Agg')
from tkinter import *
import cv2
from PIL import ImageTk, Image
from  tkinter.filedialog import askopenfilename
from fastai.conv_learner import *
from fastai.dataset import *
from fastai.models.resnet import vgg_resnet50

import json

In [2]:
PATH = Path('C:/Users/Pumpkin/Downloads/background_seg/')
print(type(PATH))

tr_path = Path('C:/Users/Pumpkin/Downloads/background_seg/total_image')
msk_path = Path('C:/Users/Pumpkin/Downloads/background_seg/total_mask')
train_dir = os.listdir('C:/Users/Pumpkin/Downloads/background_seg/total_image')
msk_dir = os.listdir('C:/Users/Pumpkin/Downloads/background_seg/total_mask')

tr_path_test = Path('C:/Users/Pumpkin/Downloads/background_seg/test_xie')
#msk_path_test = Path('project/test_mask')
train_dir_test = os.listdir('C:/Users/Pumpkin/Downloads/background_seg/test_xie')
#msk_dir_test = os.listdir('project/test_mask')

ims = [open_image(tr_path/train_dir[i]) for i in range(16)]
im_masks = [open_image(msk_path/msk_dir[i]) for i in range(16)]


x_names = np.array([tr_path/o for o in train_dir])

y_names = np.array([msk_path/f'{o[:-4]}.png' for o in train_dir])

val_idxs = list(range(300))
((val_x,trn_x),(val_y,trn_y)) = split_by_idx(val_idxs, x_names, y_names)
print(len(val_x),len(trn_x))
print(trn_x[0])


<class 'pathlib.WindowsPath'>
300 1235
C:\Users\Pumpkin\Downloads\background_seg\total_image\000000014238.jpg


In [3]:
aug_tfms = [RandomRotate(4, tfm_y=TfmType.CLASS),
            RandomFlip(tfm_y=TfmType.CLASS),
            RandomLighting(0.05, 0.05, tfm_y=TfmType.CLASS)]

In [4]:
class MatchedFilesDataset(FilesDataset):
    def __init__(self, fnames, y, transform, path):
        self.y=y
        assert(len(fnames)==len(y))
        super().__init__(fnames, transform, path)
    def get_y(self, i): return open_image(os.path.join(self.path, self.y[i]))
    def get_c(self): return 0
    
    
# def get_base():
#     layers = cut_model(f(True), cut)
#     return nn.Sequential(*layers)

def dice(pred, targs):
    pred = (pred>0).float()
    return 2. * (pred*targs).sum() / (pred+targs).sum()


class StdUpsample(nn.Module):
    def __init__(self, nin, nout):
        super().__init__()
        self.conv = nn.ConvTranspose2d(nin, nout, 2, stride=2)
        self.bn = nn.BatchNorm2d(nout)
        
    def forward(self, x): return self.bn(F.relu(self.conv(x)))
    

class Upsample34(nn.Module):
    def __init__(self, rn):
        super().__init__()
        self.rn = rn
        self.features = nn.Sequential(
            rn, nn.ReLU(),
            StdUpsample(512,256),
            StdUpsample(256,256),
            StdUpsample(256,256),
            StdUpsample(256,256),
            nn.ConvTranspose2d(256, 1, 2, stride=2))
        
    def forward(self,x): return self.features(x)[:,0]
    
class UpsampleModel():
    def __init__(self,model,name='upsample'):
        self.model,self.name = model,name

    def get_layer_groups(self, precompute):
        lgs = list(split_by_idxs(children(self.model.rn), [lr_cut]))
        return lgs + [children(self.model.features)[1:]]
    
    
class SaveFeatures():
    features=None
    def __init__(self, m): self.hook = m.register_forward_hook(self.hook_fn)
    def hook_fn(self, module, input, output): self.features = output
    def remove(self): self.hook.remove()

        
class UnetBlock(nn.Module):
    def __init__(self, up_in, x_in, n_out):
        super().__init__()
        up_out = x_out = n_out//2
        self.x_conv  = nn.Conv2d(x_in,  x_out,  1)
        self.tr_conv = nn.ConvTranspose2d(up_in, up_out, 2, stride=2)
        self.bn = nn.BatchNorm2d(n_out)
        
    def forward(self, up_p, x_p):
        up_p = self.tr_conv(up_p)
        x_p = self.x_conv(x_p)
        cat_p = torch.cat([up_p,x_p], dim=1)
        return self.bn(F.relu(cat_p))
    
class Unet34(nn.Module):
    def __init__(self, rn):
        super().__init__()
        self.rn = rn
        self.sfs = [SaveFeatures(rn[i]) for i in [2,4,5,6]]
        self.up1 = UnetBlock(512,256,256)
        self.up2 = UnetBlock(256,128,256)
        self.up3 = UnetBlock(256,64,256)
        self.up4 = UnetBlock(256,64,256)
        self.up5 = nn.ConvTranspose2d(256, 1, 2, stride=2)
        
    def forward(self,x):
        x = F.relu(self.rn(x))
        x = self.up1(x, self.sfs[3].features)
        x = self.up2(x, self.sfs[2].features)
        x = self.up3(x, self.sfs[1].features)
        x = self.up4(x, self.sfs[0].features)
        x = self.up5(x)
        return x[:,0]
    
    def close(self):
        for sf in self.sfs: sf.remove()

            
class UnetModel():
    def __init__(self,model,lr_cut,name='unet'):
        self.model,self.name = model,name
        self.lr_cut = lr_cut

    def get_layer_groups(self, precompute):
        lgs = list(split_by_idxs(children(self.model.rn), [self.lr_cut]))
        return lgs + [children(self.model)[1:]]
    
def dice(pred, targs):
    pred = (pred>0).float()
    return 2. * (pred*targs).sum() / (pred+targs).sum()

def show_img(im, figsize=None, ax=None, alpha=None):
    if not ax: fig,ax = plt.subplots(figsize=figsize)
    ax.imshow(im, alpha=alpha)
    ax.set_axis_off()
    return ax

In [5]:
sz = 128
bs = 8

In [6]:
tfms = tfms_from_model(resnet34, sz, crop_type=CropType.NO, tfm_y=TfmType.CLASS, aug_tfms=aug_tfms)

In [ ]:
class Page_one:
    def __init__(self,root,path,labelfont,title):
        self.root = root
        self.path = path
        self.labelfont = labelfont
        self.title = title
        self.w1 = Label(self.root, text = self.title)
        self.w1.config(bg='black', fg='yellow')
        self.w1.config(font= self.labelfont)
        self.w1.config(height=3, width=20)
        self.w1.pack(expand=YES, fill=BOTH)

        # background
        back_img = PhotoImage(file=self.path)
        self.panel = Label(self.root, image=back_img)

        self.panel.back_img = back_img  # you must give the reference to this
        self.panel.pack()

        # upload file button
        self.button_open = Button(root, text='Ready!', command=self.create_window, height=2, width=6)
        self.button_open.pack()

    def create_window(self):
        self.root.withdraw()
        self.window = Toplevel(self.root)
        bb = Page_two(self.window)


class Page_two():
    def __init__(self, root):
        self.root = root
        self.root.geometry('512x512')
        self.button_upload = Button(self.root, text='1.Open', command=self.OpenFile, height=2, width=12).grid(row=2,column=2)
        self.button_change = Button(self.root,text = '2.Begin remove', command = self.ChangeFile,height = 2, width = 15).grid(row=2,column=4)
        self.button_check = Button(self.root,text = '3.Check', command = self.CheckFile,height = 2, width = 12).grid(row=4,column=2)
        self.button_check = Button(self.root,text = '4.Change background', command = self.Back_Change,height = 2, width = 15).grid(row=4,column=4)
        
        
    def OpenFile(self):
        self.name = askopenfilename(initialdir="C:/Users/Pumpkin/Desktop",
                                   filetypes=(("jpeg files", "*.png"), ("all files", "*.*")),
                                   title="Choose a image."
                                   )
        print(self.name)
        input_img = PhotoImage(file = self.name)
        self.panel = Label(self.root, image=input_img)
        self.panel.back_img = input_img  # you must give the reference to this
        self.panel.grid()
    def get_base(self):
        layers = cut_model(self.f(True), self.cut)
        return nn.Sequential(*layers)

    def ChangeFile(self):
       #print(self.name)
        pp = Path(self.name)
        x_test = np.array([pp])
#         print(pp)
#         print(type(pp))
        datasets = ImageData.get_ds(MatchedFilesDataset, (trn_x,trn_y), (val_x,val_y),tfms,test=(x_test,x_test), path=PATH)
        md = ImageData(PATH, datasets, bs, num_workers=4, classes=None)
        denorm = md.trn_ds.denorm
        self.f = resnet34
        self.cut,self.lr_cut = model_meta[self.f]
        self.m_base = self.get_base()
        self.m = to_gpu(Unet34(self.m_base))
        models = UnetModel(self.m,self.lr_cut)
        learn = ConvLearner(md, models)
        learn.opt_fn=optim.Adam
        learn.crit=nn.BCEWithLogitsLoss()
        learn.metrics=[accuracy_thresh(0.5),dice]
        learn.load('final_best_128urn-0')
        x,y = next(iter(md.test_dl))
        self.py = to_np(learn.model(V(x)))
        self.pyy = self.py[0]>0
        #print(self.pyy.shape)
        #print(self.pyy)
        #self.show_img();
        self.w = Label(self.root, text="Finish transfering! Please click CHECK.")
        self.w.grid(row=5,column=2)


        
    def CheckFile(self):
        self.img_arr = 255*self.pyy
        cv2.imwrite('img.png',self.img_arr)
        input_img = PhotoImage(file = 'C:/Users/Pumpkin/fastai/old/img.png')
        self.panel2 = Label(self.root, image= input_img)
        self.panel2.back_img = input_img  # you must give the reference to this
        self.panel2.grid()
        
    def Back_Change(self):
        self.w.destroy()
        self.button_sea = Button(self.root,text = 'seaside', command = self.click_sea,height = 2, width = 8).grid(row=5,column=2)
        self.button_mountain = Button(self.root,text = 'mountain', command = self.click_city,height = 2, width = 8).grid(row=6,column=2)
        
    def click_sea(self):
        self.sea_img = cv2.imread('C:/Users/Pumpkin/fastai/old/sea.png')
        self.imgg = cv2.imread(self.name)
        for i in range(3):
            self.imgg[:,:,i] = self.imgg[:,:,i]*self.pyy
        for i in range(3):
            self.sea_img[:,:,i] = self.sea_img[:,:,i]*(~self.pyy)
        self.total = self.sea_img + self.imgg
        cv2.imwrite('total1.png',self.total)
        input_img = PhotoImage(file = 'C:/Users/Pumpkin/fastai/old/total1.png')
        self.panel3 = Label(self.root, image= input_img)
        self.panel3.back_img = input_img  # you must give the reference to this
        self.panel3.grid(row=6,column=5)
        
    def click_city(self):
        self.city_img = cv2.imread('C:/Users/Pumpkin/fastai/old/mountain.png')
        self.imgg = cv2.imread(self.name)
        for i in range(3):
            self.imgg[:,:,i] = self.imgg[:,:,i]*self.pyy
        for i in range(3):
            self.city_img[:,:,i] = self.city_img[:,:,i]*(~self.pyy)
        self.total = self.city_img + self.imgg
        cv2.imwrite('total2.png',self.total)
        input_img = PhotoImage(file = 'C:/Users/Pumpkin/fastai/old/total2.png')
        self.panel4 = Label(self.root, image= input_img)
        self.panel4.back_img = input_img  # you must give the reference to this
        self.panel4.grid(row=6,column=5)
        
        
        
        

        

In [ ]:
if __name__ == '__main__':
    root = Tk()
    path = 'C:/Users/Pumpkin/Desktop/iteraction/2.png'
    back_img = PhotoImage(file = path)
    labelfont = ('times', 15, 'bold')
    title = """Welcome to GO SELFIES and have the background removal!"""
    b = Page_one(root,path,labelfont,title)
    root.mainloop()

C:/Users/Pumpkin/Downloads/background_seg/test_xie/9.png
